In [1]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent, detect_silence
from pydub.silence import split_on_silence
import numpy as np
import ffmpeg

from os import listdir
from os.path import isfile, join

#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

In [2]:
with open("./records/_records.txt") as f:
    contents = f.readlines()
onlyfiles = [f for f in listdir("./records/") if isfile(join("./records/", f))]
onlyfiles = list(filter(lambda x: ".wav" in x, onlyfiles))

In [3]:
data = []
for file in onlyfiles:
#Convert wav to audio_segment
    name = file[:file.rfind(".")]
    audio_segment = AudioSegment.from_wav("./records/" + name + ".wav")

    #normalize audio_segment to -20dBFS 
    normalized_sound = match_target_amplitude(audio_segment, -20.0)
    print("length of audio_segment = {} seconds".format(len(normalized_sound)/1000))

    #Print detected non-silent chunks, which in our case would be spoken words.
    nonsilent_data = detect_silence(normalized_sound, min_silence_len=500, silence_thresh=-25, seek_step=1)

    #convert ms to seconds
    combined = []
    print("start, stop")
    flattened = []
    for tuple in nonsilent_data:
        for value in tuple:
            flattened.append(value)

    flattened = np.array(flattened)

    res = list(filter(lambda x: name in x, contents))[0]
    mood = 0
    if "bad" in res:
        mood = 0
    if "neutral" in res:
        mood = 1
    if "good" in res:
        mood = 2
    flattened_pmeta = [mood, flattened]
    flattened_pmeta = np.array(flattened_pmeta)
    data.append(flattened_pmeta)

data = np.array(data)

length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 180.117 seconds


C:\Users\ioci\AppData\Local\Temp\ipykernel_11244\3153445208.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  flattened_pmeta = np.array(flattened_pmeta)


start, stop
length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 180.117 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop
length of audio_segment = 15.12 seconds
start, stop


In [4]:
import pandas as pd

# Convert `X` to data frame
df = pd.DataFrame(data)

# Rename columns
df.columns = ['feature_' + str(i + 1) for i in range(df.shape[1])]

# Convert the feature with lists inside to long format
x = df['feature_2'].explode().to_frame()

# Create counter by observation so we can pivot
x['observation_id'] = x.groupby(level=0).cumcount()

# Convert to dataset and rename all columns
x = x.pivot(columns='observation_id', values='feature_2').fillna(0)
x = x.add_prefix('list_element_')

# Drop `feature_5` from X
df.drop(columns='feature_2', axis=1, inplace=True)

# Concatenate X and x together
df = pd.concat([df, x], axis=1)

In [5]:
df.head(20)

,feature_1,list_element_0,list_element_1,list_element_2,list_element_3,list_element_4,list_element_5,list_element_6,list_element_7,list_element_8,...,list_element_64,list_element_65,list_element_66,list_element_67,list_element_68,list_element_69,list_element_70,list_element_71,list_element_72,list_element_73
0,2,0,5770,8558,9373,9688,46187,46356,75305,76033,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2356,2378,7516,8001,15322,15752,16366,20933,...,0,0,0,0,0,0,0,0,0,0
2,2,0,3102,3198,4254,4273,8922,9096,10213,11002,...,0,0,0,0,0,0,0,0,0,0
3,2,24,8053,13896,16239,16266,17692,18948,59626,59627,...,0,0,0,0,0,0,0,0,0,0
4,2,1740,6695,9895,10686,11318,14081,14116,16081,19445,...,161017,161763,168230,169672,170112,170867,171154,171719,172880,180003
5,2,0,661,2324,3023,5375,6094,6435,8369,9920,...,0,0,0,0,0,0,0,0,0,0
6,1,0,11768,13462,14613,16414,17437,17533,19527,20970,...,0,0,0,0,0,0,0,0,0,0
7,0,920,8584,8720,9674,9705,12797,12827,25982,26066,...,0,0,0,0,0,0,0,0,0,0
8,1,658,2928,3738,4464,4575,12190,12282,15120,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,3615,6289,13969,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df.drop(["feature_1"],axis=1)
y = df["feature_1"]
y=y.astype('int')

In [37]:
# Carry on as before
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 44)

In [38]:
from sklearn.svm import SVC
model  = SVC()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
score = model.score(X_test, y_test)
score

0.5

In [39]:
from sklearn.ensemble import GradientBoostingClassifier
model  = GradientBoostingClassifier()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
score = model.score(X_test, y_test)
score

0.3333333333333333